 # BLS Data Cleaning

In [7]:
pip install impyute

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
pd.options.display.max_columns = None

import numpy as np
import impyute as impy
import os
oes_files = sorted([file for file in os.listdir('/content/drive/MyDrive/MADS Capstone Team 23/Data/raw/bls_income') if 'MSA_M' in file])

BLS_path = '/content/drive/MyDrive/MADS Capstone Team 23/Data/raw/bls_income'
metros = pd.read_json('/content/drive/MyDrive/MADS Capstone Team 23/Data/processed/metro_areas.json', orient='records', lines =True)


#MSA nubers updated in 2005 and 2018 couldn't nail down document to automate this so handcode required
catch_metro_code_change_ovt = [
    {'metro_code':'METRO35620','area_code':5600,'metro_name':'New York-Newark-Jersey City, NY-NJ-PA Metro Area'},
    {'metro_code':'METRO35620','area_code':35614,'metro_name':'New York-Newark-Jersey City, NY-NJ-PA Metro Area'},
    {'metro_code':'METRO31080','area_code':4480,'metro_name':'Los Angeles-Long Beach-Anaheim, CA Metro Area'},
    {'metro_code':'METRO31080','area_code':31084,'metro_name':'Los Angeles-Long Beach-Anaheim, CA Metro Area'},
    {'metro_code':'METRO16980','area_code':1600,'metro_name':'Chicago-Naperville-Elgin, IL-IN-WI Metro Area'},
    {'metro_code':'METRO16980','area_code':16974,'metro_name':'Chicago-Naperville-Elgin, IL-IN-WI Metro Area'},
    {'metro_code':'METRO19100','area_code':1920,'metro_name':'Dallas-Fort Worth-Arlington, TX Metro Area'},
    {'metro_code':'METRO19100','area_code':19124,'metro_name':'Dallas-Fort Worth-Arlington, TX Metro Area'},
    {'metro_code':'METRO26420','area_code':3360,'metro_name':'Houston-The Woodlands-Sugar Land, TX Metro Area'},
    {'metro_code':'METRO26420','area_code':26420,'metro_name':'Houston-The Woodlands-Sugar Land, TX Metro Area'},
    {'metro_code':'METRO47900','area_code':8840,'metro_name':'Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area'},
    {'metro_code':'METRO47900','area_code':47894,'metro_name':'Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area'},
    {'metro_code':'METRO37980','area_code':6160,'metro_name':'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area'},
    {'metro_code':'METRO37980','area_code':37964,'metro_name':'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Metro Area'},
    {'metro_code':'METRO12060','area_code':520,'metro_name':'Atlanta-Sandy Springs-Alpharetta, GA Metro Area'},
    {'metro_code':'METRO12060','area_code':12060,'metro_name':'Atlanta-Sandy Springs-Alpharetta, GA Metro Area'},
    {'metro_code':'METRO33100','area_code':5000,'metro_name':'Miami-Fort Lauderdale-Pompano Beach, FL Metro'},
    {'metro_code':'METRO33100','area_code':33124,'metro_name':'Miami-Fort Lauderdale-Pompano Beach, FL Metro'},
    {'metro_code':'METRO38060','area_code':6200,'metro_name':'Phoenix-Mesa-Chandler, AZ Metro Area'},
    {'metro_code':'METRO38060','area_code':38060,'metro_name':'Phoenix-Mesa-Chandler, AZ Metro Area'},
    {'metro_code':'METRO14460','area_code':1120,'metro_name':'Boston-Cambridge-Newton, MA-NH Metro Area'},
    {'metro_code':'METRO14460','area_code':71654,'metro_name':'Boston-Cambridge-Newton, MA-NH Metro Area'},
    {'metro_code':'METRO40140','area_code':6780,'metro_name':'Riverside-San Bernardino-Ontario, CA Metro Area'},
    {'metro_code':'METRO40140','area_code':40140,'metro_name':'Riverside-San Bernardino-Ontario, CA Metro Area'},
    {'metro_code':'METRO41860','area_code':7360,'metro_name':'San Francisco-Oakland-Berkeley, CA Metro Area'},
    {'metro_code':'METRO41860','area_code':41884,'metro_name':'San Francisco-Oakland-Berkeley, CA Metro Area'},
    {'metro_code':'METRO19820','area_code':2160,'metro_name':'Detroit-Warren-Dearborn, MI Metro Area'},
    {'metro_code':'METRO19820','area_code':19804,'metro_name':'Detroit-Warren-Dearborn, MI Metro Area'},
    {'metro_code':'METRO42660','area_code':7600,'metro_name':'Seattle-Tacoma-Bellevue, WA Metro Area'},
    {'metro_code':'METRO42660','area_code':42644,'metro_name':'Seattle-Tacoma-Bellevue, WA Metro Area'}

]

metros = pd.concat([metros,pd.DataFrame(catch_metro_code_change_ovt)])

In [9]:
metros

,metro_code,area_code,metro_name
0,METRO38060,6200,"Phoenix-Mesa-Chandler, AZ Metro Area"
1,METRO41860,5775,"San Francisco-Oakland-Berkeley, CA Metro Area"
2,METRO31080,4480,"Los Angeles-Long Beach-Anaheim, CA Metro Area"
3,METRO41860,7360,"San Francisco-Oakland-Berkeley, CA Metro Area"
4,METRO31080,5945,"Los Angeles-Long Beach-Anaheim, CA Metro Area"
...,...,...,...
25,METRO41860,41884,"San Francisco-Oakland-Berkeley, CA Metro Area"
26,METRO19820,2160,"Detroit-Warren-Dearborn, MI Metro Area"
27,METRO19820,19804,"Detroit-Warren-Dearborn, MI Metro Area"
28,METRO42660,7600,"Seattle-Tacoma-Bellevue, WA Metro Area"


In [11]:
name_consistency_map = {
    'occ_titl' : 'occ_title',
    'a_wpct10' : 'a_pct10',
    'a_wpct25' : 'a_pct25',
    'a_wpct75' : 'a_pct75',
    'a_wpct90' : 'a_pct90',
    'h_wpct10' : 'h_pct10',
    'h_wpct25' : 'h_pct25',
    'h_wpct75' : 'h_pct75',
    'h_wpct90' : 'h_pct90',
    'group'    : 'o_group',
    'occ_group': 'o_group',
    'area_name': 'area_title'

}

In [12]:
final_columns = ['year', 'area', 'area_title', 'occ_code', 'o_group', 'occ_title', 'tot_emp', 'a_mean', 'a_median',  'h_mean', 'h_median' ]
oes_frames=[]
prior_columns = []
prior_file=''
for file in oes_files:
    if len(prior_file)==0:
        prior_file = file
    try:
        oes_df=pd.read_excel(BLS_path + file )
        oes_df.columns = [x.lower() for x in oes_df.columns]
        oes_df = oes_df.rename(columns=name_consistency_map)
        oes_df['year'] = int(file[5:9])
        if (oes_df['occ_title'] == 'Industry Total').any():
            oes_df['occ_title'] = oes_df['occ_title'].replace('Industry Total', 'All Occupations')

        oes_df['o_group'] = oes_df['o_group'].str.strip()
        mask1 = oes_df['occ_title'] == 'All Occupations'
        oes_df.loc[mask1, 'o_group'] = oes_df.loc[mask1, 'o_group'].replace('', 'total')
        mask2 = oes_df['o_group'] == ''
        oes_df.loc[mask2, 'o_group'] = oes_df.loc[mask2, 'o_group'].replace('', 'detailed')
        oes_df['o_group'] = oes_df['o_group'].fillna('detailed')
        oes_df = oes_df[final_columns]
        current_columns = list(oes_df.columns)
        oes_df = oes_df[oes_df['o_group']!='detailed']



        print(f'Creation successful for {file}')
        if len(prior_columns) ==0:
            prior_columns = current_columns
            print(prior_columns)
        if len(prior_columns) != len(current_columns):
            print(f'    {file} differs from {prior_file} in number of fields')
        c_set=set(current_columns)
        p_set=set(prior_columns)
        super_set = p_set.union(c_set)
        if len(super_set-c_set) != 0 or len(super_set-p_set)!=0:
            if len(super_set-c_set) != 0:
                print(f'        {prior_file} contains fields not found in {file}')
                print(f'            {sorted(list(super_set-c_set))}')
            if  len(super_set-p_set) != 0:
                print(f'        {file} contains fields not found in {prior_file}')
                print(f'            {sorted(list(super_set-p_set))}')
        prior_columns = current_columns
        prior_file = file

        oes_frames.append(oes_df)

    except:
        print(f'Creation failed for {file}')



#year 2000 does not have occupation total

Creation failed for MSA_M2000_dl_1.xls
Creation failed for MSA_M2000_dl_2.xls
Creation failed for MSA_M2001_dl_1.xls
Creation failed for MSA_M2001_dl_2.xls
Creation failed for MSA_M2001_dl_3.xls
Creation failed for MSA_M2002_dl_1.xls
Creation failed for MSA_M2002_dl_2.xls
Creation failed for MSA_M2003_dl_1.xls
Creation failed for MSA_M2003_dl_2.xls
Creation failed for MSA_M2004_dl_1.xls
Creation failed for MSA_M2004_dl_2.xls
Creation failed for MSA_M2004_dl_3.xls
Creation failed for MSA_M2005_dl_1.xls
Creation failed for MSA_M2005_dl_2.xls
Creation failed for MSA_M2005_dl_3.xls
Creation failed for MSA_M2006_dl_1.xls
Creation failed for MSA_M2006_dl_2.xls
Creation failed for MSA_M2006_dl_3.xls
Creation failed for MSA_M2007_dl_1.xls
Creation failed for MSA_M2007_dl_2.xls
Creation failed for MSA_M2007_dl_3.xls
Creation failed for MSA_M2008_dl_1.xls
Creation failed for MSA_M2008_dl_2.xls
Creation failed for MSA_M2008_dl_3.xls
Creation failed for MSA_M2009_dl_1.xls
Creation failed for MSA_M

In [13]:
oes_data=pd.DataFrame()
for frame in oes_frames:
    frame.reset_index(drop = True, inplace = True)
    #frame.fillna(value = np.nan, inplace = True)
for frame in oes_frames:
    oes_data=oes_data.append(frame, sort=False)
    oes_data = oes_data.reset_index(drop=True)
oes_data=oes_data.drop(['level_0', 'index'],axis =1)
oes_data

KeyError: "['level_0', 'index'] not found in axis"

In [ ]:
#uncomment for sanity check of years
#oes_data[(oes_data.area_title.str.contains('Seattle'))&(oes_data.occ_title.str.contains('Man'))][['area_title', 'area', 'year']].head(50)

In [ ]:
#prep codes and metro names
metros['short_code'] = metros['metro_code'].str[5:].astype(int)
metros['metro_name'] = metros['metro_name'].str.replace(' Metro Area','')
metros['metro_name'] = metros['metro_name'].str.replace(' Metro','')

#some overlap of major metro codes and sub metro codes. filter her
major_areas= list(metros['short_code'].unique())
minor_areas=[int(ele) for ele in list(set(metros['area_code'].unique())-set(major_areas)) ]
areas_filter = list(set(major_areas).union(set(minor_areas)))

oes_data=oes_data[oes_data['area'].isin(areas_filter)]
oes_data.head()


In [ ]:
#prep loop aggreagation of sub areas
minor_filters = oes_data.copy()[['year', 'occ_code', 'o_group','occ_title']].drop_duplicates().values.tolist()
major_filters = metros.copy()[['metro_name', 'short_code']].drop_duplicates().values.tolist()
minor_area_df = oes_data.copy()
minor_area_df = minor_area_df[~minor_area_df['area'].isin(major_areas)]
records = []

for year, code, group, title in minor_filters:
    df = minor_area_df.copy()
    df = df[(df['year'] == year) & (df['occ_code'] == code) & (df['o_group'] == group) & (df['occ_title'] == title)]
    for metro_name, metro_code in major_filters:
        mdf = metros[metros['short_code'] == metro_code].copy()
        df1 = df.copy()
        df1 = df1[df1['area'].isin(list(mdf['area_code'].unique()))]

        if len(df1) == 0:
            continue

        # Select columns that need to be numeric and fill NaN with the column's mean value
        numeric_columns = ['tot_emp', 'a_mean', 'h_mean', 'a_median', 'h_median']
        for col in numeric_columns:
            df1[col] = pd.to_numeric(df1[col], errors='coerce')
            df1[col].fillna(df1[col].mean(), inplace=True)

        employee_total = df1['tot_emp'].sum()
        df1['a_mean'] = df1['a_mean'] * df1['tot_emp']
        df1['h_mean'] = df1['h_mean'] * df1['tot_emp']
        df1['a_median'] = df1['a_median'] * df1['tot_emp']
        df1['h_median'] = df1['h_median'] * df1['tot_emp']

        if employee_total == 0:
            continue

        record = {
            'year': year,
            'area': metro_code,
            'area_title': metro_name,
            'occ_code': code,
            'o_group': group,
            'occ_title': title,
            'tot_emp': str(employee_total),
            'a_mean': str(df1['a_mean'].sum() / employee_total),
            'a_median': str(df1['a_median'].sum() / employee_total),
            'h_mean': str(np.round(df1['h_mean'].sum() / employee_total, 2)),
            'h_median': str(np.round(df1['h_median'].sum() / employee_total, 2)),
        }

        records.append(record)

minor_df = pd.DataFrame(records)
minor_df


In [ ]:
major_df= oes_data.copy()
major_df= major_df[major_df['area'].isin(major_areas)]
major_df[(major_df.area_title.str.contains('Seattle'))&(major_df.occ_title.str.contains('Man'))][['area_title', 'area', 'year']].head(50)
oes_final= minor_df.append(major_df)
oes_final.head()

In [ ]:
#second sanity check
#new_data[(new_data.area_title.str.contains('Seattle'))&(new_data.occ_title.str.contains('Man'))][['area_title', 'area', 'year']].head(50)

In [ ]:
oes_final.to_json('/content/drive/MyDrive/MADS Capstone Team 23/Data/processed/bls_income/BLS_income_data.json', orient='records', lines = True)